## Adversarial AutoEncoder (AAE)
Adversarial AutoEncoder (AAE) is an idea of blending `AutoEncoder` with the `adversarial loss` concept. It is same as `Variational AutoEncoder (VAE)` except that it uses `adversarial loss` to regularize the latent code instead of the `KL-Divergence` that `VAE` uses.

You can understand the whole `AAE` on this medium post: [Adversarial Auto Encoder (AAE)](https://medium.com/vitrox-publication/adversarial-auto-encoder-aae-a3fc86f71758)

<img src="https://miro.medium.com/max/700/1*uJnxUa9voPETKJTGudH46w.png"/>

<img src="https://miro.medium.com/max/700/1*cPKheE0hF0ibIW-dvSCqmQ.png"/>

### Implementation of AAE

In [1]:
import os
import numpy as np
import math
import itertools

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
os.makedirs('images', exist_ok=True)

In [3]:
n_epochs = 100
batch_size = 64
lr = 0.0002

# betas
b1 = 0.5
b2 = 0.999

n_cpu = 8
latent_dim = 10
img_size = 32
channels = 1
sample_interval = 400

In [4]:
img_shape = (channels, img_size, img_size)

# Device
cuda = True if torch.cuda.is_available() else False

In [5]:
cuda

True

In [14]:
def reparameterization(mu, logvar):
    std = torch.exp(logvar / 2)
    sampled_z = Variable(Tensor(np.random.normal(0, 1, (mu.size(0), latent_dim))))
    z = sampled_z * std + mu
    return z

`Encoder` Module

In [8]:
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(int(np.prod(img_shape)), 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(0.2, inplace=True)
        )

        self.mu = nn.Linear(512, latent_dim)
        self.logvar = nn.Linear(512, latent_dim)

    def forward(self, img):
        img_flat = img.view(img.shape[0], -1)
        x = self.model(img_flat)
        mu = self.mu(x)
        logvar = self.logvar(x)
        z = reparameterization(mu, logvar)
        return z

In [9]:
# Decoder Class
class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(latent_dim, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, int(np.prod(img_shape))),
            nn.Tanh(),
        )

    def forward(self, z):
        img_flat = self.model(z)
        img = img_flat.view(img_flat.shape[0], *img_shape)
        return img

In [10]:
# Discriminator
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(latent_dim, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid(),
        )

    def forward(self, z):
        validity = self.model(z)
        return validity

In [11]:
# Use Binary Cross-entropy loss
adversarial_loss = torch.nn.BCELoss()
pixelwise_loss = torch.nn.L1Loss()

# Initialize Encoder, Decoder, Discriminator
encoder = Encoder()
decoder = Decoder()
discriminator = Discriminator()

In [12]:
if cuda:
    encoder.cuda()
    decoder.cuda()
    discriminator.cuda()

    adversarial_loss.cuda()
    pixelwise_loss.cuda()

In [13]:
# Configure DataFolder
os.makedirs('/data/mnist', exist_ok=True)

dataloader = DataLoader(
    datasets.MNIST(
        '/data/mnist',
        train = True,
        download = True,
        transform = transforms.Compose(
            [transforms.Resize(img_size), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
        ),
    ),
    batch_size =  batch_size,
    shuffle = True
)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting /data/mnist/MNIST/raw/train-images-idx3-ubyte.gz to /data/mnist/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting /data/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to /data/mnist/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting /data/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to /data/mnist/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting /data/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to /data/mnist/MNIST/raw



In [15]:
# Optimizers
optimizer_G = torch.optim.Adam(
    itertools.chain(encoder.parameters(), decoder.parameters()), lr=lr, betas=(b1, b2))

optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(b1, b2))

In [16]:
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

In [17]:
Tensor

torch.cuda.FloatTensor

In [18]:
def sample_image(n_row, batches_done):
    """Saves a grid of generated digits"""
    # Sample noise
    z = Variable(Tensor(np.random.normal(0, 1, (n_row ** 2, latent_dim))))
    gen_imgs = decoder(z)
    save_image(gen_imgs.data, "images/%d.png" % batches_done, nrow=n_row, normalize=True)

In [19]:
# ----------
#  Training
# ----------

for epoch in range(n_epochs):
    for i, (imgs, _) in enumerate(dataloader):

        # Adversarial ground truths
        valid = Variable(Tensor(imgs.shape[0], 1).fill_(1.0), requires_grad=False)
        fake = Variable(Tensor(imgs.shape[0], 1).fill_(0.0), requires_grad=False)

        # Configure input
        real_imgs = Variable(imgs.type(Tensor))

        # -----------------
        #  Train Generator
        # -----------------

        optimizer_G.zero_grad()

        encoded_imgs = encoder(real_imgs)
        decoded_imgs = decoder(encoded_imgs)

        # Loss measures generator's ability to fool the discriminator
        g_loss = 0.001 * adversarial_loss(discriminator(encoded_imgs), valid) + 0.999 * pixelwise_loss(
            decoded_imgs, real_imgs
        )

        g_loss.backward()
        optimizer_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Sample noise as discriminator ground truth
        z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], latent_dim))))

        # Measure discriminator's ability to classify real from generated samples
        real_loss = adversarial_loss(discriminator(z), valid)
        fake_loss = adversarial_loss(discriminator(encoded_imgs.detach()), fake)
        d_loss = 0.5 * (real_loss + fake_loss)

        d_loss.backward()
        optimizer_D.step()

        print(
            "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
            % (epoch, n_epochs, i, len(dataloader), d_loss.item(), g_loss.item())
        )

        batches_done = epoch * len(dataloader) + i
        if batches_done % sample_interval == 0:
            sample_image(n_row=10, batches_done=batches_done)

Streaming output truncated to the last 5000 lines.
[Epoch 94/100] [Batch 628/938] [D loss: 0.369650] [G loss: 0.068595]
[Epoch 94/100] [Batch 629/938] [D loss: 0.338291] [G loss: 0.067126]
[Epoch 94/100] [Batch 630/938] [D loss: 0.409830] [G loss: 0.064632]
[Epoch 94/100] [Batch 631/938] [D loss: 0.343850] [G loss: 0.068055]
[Epoch 94/100] [Batch 632/938] [D loss: 0.460602] [G loss: 0.073828]
[Epoch 94/100] [Batch 633/938] [D loss: 0.281783] [G loss: 0.066037]
[Epoch 94/100] [Batch 634/938] [D loss: 0.301106] [G loss: 0.071445]
[Epoch 94/100] [Batch 635/938] [D loss: 0.397920] [G loss: 0.078212]
[Epoch 94/100] [Batch 636/938] [D loss: 0.304710] [G loss: 0.069903]
[Epoch 94/100] [Batch 637/938] [D loss: 0.354019] [G loss: 0.071488]
[Epoch 94/100] [Batch 638/938] [D loss: 0.388743] [G loss: 0.075417]
[Epoch 94/100] [Batch 639/938] [D loss: 0.339452] [G loss: 0.068877]
[Epoch 94/100] [Batch 640/938] [D loss: 0.378763] [G loss: 0.064226]
[Epoch 94/100] [Batch 641/938] [D loss: 0.279835] [G